# Observer (옵저버)

    어떠한 동작이 발생했다는 사실을 여러 객체에게 전파

## 정의

p가 있고, p.subscribers(set)가 있고, p는 p.subscribers에 s를 추가하는 동작(set.add)과 p의 어떤 동작이 수행되었을 때 p.subscribers를 순회하며 s의 메서드를 트리거하는 동작(subscriber의 인터페이스)이 구현되어 있다.

p의 동작이 트리거 역할을 해서 여러 s의 동작 실행을 유발해야 할 때 사용할 수 있는 패턴.

아래 코드에서
- dict를 사용해서 event(str):subscribers(set)을 매핑해서 여러 이벤트를 활용할 수 있도록 한 부분
- subscribers를 set으로 지정하는 부분
- 클라이언트가 활용하는 객체가 publisher 객체를 속성으로 부여하는 부분

들이 주목할 만 하다.

## 구현

In [49]:
from __future__ import annotations
from collections import defaultdict
from pathlib import Path


class EventManager:

    def __init__(self):
        self.listeners: defaultdict[str, set[EventListener]] = defaultdict(set)
    
    def subscribe(self, event_type: str, listener: EventListener):
        self.listeners[event_type].add(listener)
    
    def unsubscribe(self, event_type: str, listener: EventListener):
        self.listeners[event_type].remove(listener)
    
    def notify(self, event_type: str, data: str):
        for listener in self.listeners[event_type]:
            listener.update(data)


class EventListener:

    def update(self, file_name: str):
        ...


class LoggingListener(EventListener):

    def __init__(self, log_file_name: str, message: str):
        self.log_file_name = log_file_name
        self.message = message
    
    def update(self, file_name):
        print(f"LoggingListener : {self.message.format(file_name)} (write {self.log_file_name})")


class EmailAlertsListener(EventListener):

    def __init__(self, to_email: str, message: str):
        self.to_email = to_email
        self.message = message
    
    def update(self, file_name: str):
        print(f"EmailAlertsListener : {self.message.format(file_name)} (to {self.to_email})")


class Editor:

    def __init__(self):
        self.event_manager = EventManager()
        self.file = None
    
    def open_file(self, path: str):
        self.file = f"{path} file"
        self.event_manager.notify("open", self.file)
    
    def save_file(self):
        self.event_manager.notify("save", self.file)


editor = Editor()

logger = LoggingListener("/path/to/log.txt", "Someone has opened the file: {}")
editor.event_manager.subscribe("open", logger)

email_sender = EmailAlertsListener("admin@example.com", "Someone has changed the file: {}")
editor.event_manager.subscribe("save", email_sender)

editor.open_file("some/path/file.py")
editor.save_file()

LoggingListener : Someone has opened the file: some/path/file.py file (write /path/to/log.txt)
EmailAlertsListener : Someone has changed the file: some/path/file.py file (to admin@example.com)
